In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drw-crypto-market-prediction/sample_submission.csv
/kaggle/input/drw-crypto-market-prediction/train.parquet
/kaggle/input/drw-crypto-market-prediction/test.parquet


In [2]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from scipy.stats import pearsonr
import shap

In [13]:
import pandas as pd

# Load the parquet file into a DataFrame
df = pd.read_parquet('/kaggle/input/drw-crypto-market-prediction/train.parquet')

# Display first few rows
print(df.head())

                     bid_qty  ask_qty  buy_qty  sell_qty   volume        X1  \
2023-03-01 00:00:00   15.283    8.425  176.405    44.984  221.389  0.181844   
2023-03-01 00:01:00   38.590    2.336  525.846   321.950  847.796  0.489497   
2023-03-01 00:02:00    0.442   60.250  159.227   136.369  295.596  0.260121   
2023-03-01 00:03:00    4.865   21.016  335.742   124.963  460.705  0.099976   
2023-03-01 00:04:00   27.158    3.451   98.411    44.407  142.818  0.270893   

                           X2        X3        X4        X5  ...      X772  \
2023-03-01 00:00:00 -0.637860  0.006652  0.136870  0.116698  ...  0.333753   
2023-03-01 00:01:00 -0.075619  0.431594  0.522400  0.475255  ...  0.333657   
2023-03-01 00:02:00 -0.444684  0.100695  0.224729  0.203282  ...  0.333667   
2023-03-01 00:03:00 -0.666728 -0.123858  0.019197  0.014459  ...  0.333174   
2023-03-01 00:04:00 -0.325973  0.116336  0.234311  0.214073  ...  0.333171   

                         X773      X774      X775      X

In [3]:
# ----  Feature engineering with lagging ---- # 

def add_features(df):
    df['bid_ask_interaction'] = df['bid_qty'] * df['ask_qty']
    df['bid_buy_interaction'] = df['bid_qty'] * df['buy_qty']
    df['bid_sell_interaction'] = df['bid_qty'] * df['sell_qty']
    df['ask_buy_interaction'] = df['ask_qty'] * df['buy_qty']
    df['ask_sell_interaction'] = df['ask_qty'] * df['sell_qty']
    df['buy_sell_interaction'] = df['buy_qty'] * df['sell_qty']

    df['spread_indicator'] = (df['ask_qty'] - df['bid_qty']) / (df['ask_qty'] + df['bid_qty'] + 1e-8)

    df['volume_weighted_buy'] = df['buy_qty'] * df['volume']
    df['volume_weighted_sell'] = df['sell_qty'] * df['volume']
    df['volume_weighted_bid'] = df['bid_qty'] * df['volume']
    df['volume_weighted_ask'] = df['ask_qty'] * df['volume']

    df['buy_sell_ratio'] = df['buy_qty'] / (df['sell_qty'] + 1e-8)
    df['bid_ask_ratio'] = df['bid_qty'] / (df['ask_qty'] + 1e-8)

    df['order_flow_imbalance'] = (df['buy_qty'] - df['sell_qty']) / (df['volume'] + 1e-8)

    df['buying_pressure'] = df['buy_qty'] / (df['volume'] + 1e-8)
    df['selling_pressure'] = df['sell_qty'] / (df['volume'] + 1e-8)

    df['total_liquidity'] = df['bid_qty'] + df['ask_qty']
    df['liquidity_imbalance'] = (df['bid_qty'] - df['ask_qty']) / (df['total_liquidity'] + 1e-8)
    df['relative_spread'] = (df['ask_qty'] - df['bid_qty']) / (df['volume'] + 1e-8)

    df['trade_intensity'] = (df['buy_qty'] + df['sell_qty']) / (df['volume'] + 1e-8)
    df['avg_trade_size'] = df['volume'] / (df['buy_qty'] + df['sell_qty'] + 1e-8)
    df['net_trade_flow'] = (df['buy_qty'] - df['sell_qty']) / (df['buy_qty'] + df['sell_qty'] + 1e-8)

    df['depth_ratio'] = df['total_liquidity'] / (df['volume'] + 1e-8)
    df['volume_participation'] = (df['buy_qty'] + df['sell_qty']) / (df['total_liquidity'] + 1e-8)
    df['market_activity'] = df['volume'] * df['total_liquidity']

    df['effective_spread_proxy'] = np.abs(df['buy_qty'] - df['sell_qty']) / (df['volume'] + 1e-8)
    df['realized_volatility_proxy'] = np.abs(df['order_flow_imbalance']) * df['volume']

    df['normalized_buy_volume'] = df['buy_qty'] / (df['bid_qty'] + 1e-8)
    df['normalized_sell_volume'] = df['sell_qty'] / (df['ask_qty'] + 1e-8)

    df['liquidity_adjusted_imbalance'] = df['order_flow_imbalance'] * df['depth_ratio']
    df['pressure_spread_interaction'] = df['buying_pressure'] * df['spread_indicator']

    # Trade Direction Ratio（成交主导方向）
    df['trade_direction_ratio'] = df['buy_qty'] / (df['buy_qty'] + df['sell_qty'] + 1e-8)
    
    # Net Buying Volume（净主动买入量）
    df['net_buy_volume'] = df['buy_qty'] - df['sell_qty']
    
    # Bid/Ask Skew（挂单偏移度）
    df['bid_skew'] = df['bid_qty'] / (df['bid_qty'] + df['ask_qty'] + 1e-8)
    df['ask_skew'] = df['ask_qty'] / (df['bid_qty'] + df['ask_qty'] + 1e-8)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(0, inplace=True)

    return df

In [4]:
class Config:
    TRAIN_PATH = "/kaggle/input/drw-crypto-market-prediction/train.parquet"

    TEST_PATH = "/kaggle/input/drw-crypto-market-prediction/test.parquet"
    SUBMISSION_PATH = "/kaggle/input/drw-crypto-market-prediction/sample_submission.csv"

    FEATURES = [
    "X752",
    "X287",
    "X298",
    "X759",
    "X302",
    "X55",
    "X56",
    "X52",
    "X303",
    "X51",
    "bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume",'bid_ask_interaction',
    'bid_buy_interaction', 'bid_sell_interaction', 'ask_buy_interaction', 'ask_sell_interaction', 'buy_sell_interaction',
    'spread_indicator',
    'volume_weighted_buy', 'volume_weighted_sell', 'volume_weighted_bid', 'volume_weighted_ask',
    'buy_sell_ratio', 'bid_ask_ratio',
    'order_flow_imbalance',
    'buying_pressure', 'selling_pressure',
    'total_liquidity', 'liquidity_imbalance', 'relative_spread',
    'trade_intensity', 'avg_trade_size', 'net_trade_flow',
    'depth_ratio', 'volume_participation', 'market_activity',
    'effective_spread_proxy', 'realized_volatility_proxy',
    'normalized_buy_volume', 'normalized_sell_volume',
    'liquidity_adjusted_imbalance', 'pressure_spread_interaction', 
    'trade_direction_ratio', 'net_buy_volume', 'bid_skew' , 'ask_skew'
    ]

    LABEL_COLUMN = "label"
    N_FOLDS = 3
    RANDOM_STATE = 42

In [5]:
XGB_PARAMS ={'tree_method': 'hist', 'device': 'cpu','n_jobs': -1,
            'random_state': Config.RANDOM_STATE,
             'colsample_bytree': 0.4111224922845363, 'colsample_bynode': 0.28869302181383194,
             'gamma': 1.4665430311056709, 'learning_rate': 0.014053505540364681, 
             'max_depth': 7, 'max_leaves': 40, 'n_estimators': 500,
             'reg_alpha': 27.791606770656145, 'reg_lambda': 84.90603428439086,
             'subsample': 0.06567}

LGBM_PARAMS = {
    "boosting_type": "gbdt",
    "colsample_bytree": 0.5625888953382505,
    "learning_rate": 0.029312951475451557,
    "min_child_samples": 63,
    "min_child_weight": 0.11456572852335424,
    "n_estimators": 126,
    "n_jobs": -1,
    "num_leaves": 37,
    "random_state": 42,
    "reg_alpha": 85.2476527854083,
    "reg_lambda": 99.38305361388907,
    "subsample": 0.450669817684892,
    "verbose": -1
}

LBGM_GOSS_PARAMS = {
    "boosting_type": "goss",
    "colsample_bytree": 0.34695458228489784,
    "learning_rate": 0.031023014900595287,
    "min_child_samples": 30,
    "min_child_weight": 0.4727729225033618,
    "n_estimators": 220,
    "n_jobs": -1,
    "num_leaves": 58,
    "random_state": 42,
    "reg_alpha": 38.665994901468224,
    "reg_lambda": 92.76991677464294,
    "subsample": 0.4810891284493255,
    "verbose": -1
}


LEARNERS = [
    {"name": "xgb", "Estimator": XGBRegressor, "params": XGB_PARAMS},
    {"name": "lbgm_gpbt", "Estimator":  LGBMRegressor, "params": LGBM_PARAMS},
    {"name": "lbgm_goss", "Estimator":  LGBMRegressor, "params": LBGM_GOSS_PARAMS},
]

In [6]:
# Weight the time series data

def create_time_decay_weights(n: int, decay: float = 0.9) -> np.ndarray:
    positions = np.arange(n)
    normalized = positions / (n - 1)
    weights = decay ** (1.0 - normalized)
    return weights * n / weights.sum()

In [7]:
# Loading all the data

def load_data():
    train_df = pd.read_parquet(Config.TRAIN_PATH, columns=Config.FEATURES + [Config.LABEL_COLUMN])
    test_df = pd.read_parquet(Config.TEST_PATH, columns=Config.FEATURES)
    submission_df = pd.read_csv(Config.SUBMISSION_PATH)

    train_df = feature_engineering(train_df)
    test_df = feature_engineering(test_df)
    print(f"Loaded data - Train: {train_df.shape}, Test: {test_df.shape}, Submission: {submission_df.shape}")
    return train_df.reset_index(drop=True), test_df.reset_index(drop=True), submission_df

In [8]:
Config.FEATURES =  Config.FEATURES  + ["bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume"]
Config.FEATURES = list(set(Config.FEATURES))  # remove duplicates

In [9]:
# Training few different models on different slices of the data set (K folding)

# Different slices are given as follows to enforce that it follows time
def get_model_slices(n_samples: int):
    return [
        {"name": "full_data", "cutoff": 0},
        {"name": "last_75pct", "cutoff": int(0.25 * n_samples)},
        {"name": "last_50pct", "cutoff": int(0.50 * n_samples)},
    ]

In [10]:
def train_and_evaluate(train_df, test_df):
    n_samples = len(train_df)
    model_slices = get_model_slices(n_samples)

    # Initialize models and names for tracking
    models = []
    model_names = []

    # Initialize out-of-fold (OOF) and test predictions
    oof_preds = {
        learner["name"]: {s["name"]: np.zeros(n_samples) for s in model_slices}
        for learner in LEARNERS
    }

    test_preds = {
        learner["name"]: {s["name"]: np.zeros(len(test_df)) for s in model_slices}
        for learner in LEARNERS
    }

    # Precompute full weights
    full_weights = create_time_decay_weights(n_samples)

    # K-Fold cross-validation
    kf = KFold(n_splits=Config.N_FOLDS, shuffle=False)

    for fold, (train_idx, valid_idx) in enumerate(kf.split(train_df), start=1):
        print(f"\n--- Fold {fold}/{Config.N_FOLDS} ---")

        X_valid = train_df.iloc[valid_idx][Config.FEATURES].values
        y_valid = train_df.iloc[valid_idx][Config.LABEL_COLUMN].values

        for slice_info in model_slices:
            slice_name = slice_info["name"]
            cutoff = slice_info["cutoff"]

            # Slice training data according to cutoff
            train_slice_df = train_df.iloc[cutoff:].reset_index(drop=True)
            rel_train_idx = train_idx[train_idx >= cutoff] - cutoff

            X_train = train_slice_df.iloc[rel_train_idx][Config.FEATURES].values
            y_train = train_slice_df.iloc[rel_train_idx][Config.LABEL_COLUMN].values

            # Compute sample weights for training
            if cutoff > 0:
                weights = create_time_decay_weights(len(train_slice_df))[rel_train_idx]
            else:
                weights = full_weights[train_idx]

            print(f"  Training slice: {slice_name}, samples: {len(X_train)}")

            for learner in LEARNERS:
                learner_name = learner["name"]
                model = learner["Estimator"](**learner["params"])

                # Train model
                model.fit(
                    X_train, y_train,
                    sample_weight=weights,
                    eval_set=[(X_valid, y_valid)],
                )

                models.append(model)
                model_names.append(f"fold:{fold} slice:{slice_name}")

                # Predict on validation fold
                valid_mask = valid_idx >= cutoff
                if valid_mask.any():
                    valid_rows = valid_idx[valid_mask]
                    X_valid_fold = train_df.iloc[valid_rows][Config.FEATURES].values
                    oof_preds[learner_name][slice_name][valid_rows] = model.predict(X_valid_fold)

                # For rows before cutoff, copy full_data prediction
                if cutoff > 0 and (~valid_mask).any():
                    fallback_rows = valid_idx[~valid_mask]
                    oof_preds[learner_name][slice_name][fallback_rows] = oof_preds[learner_name]["full_data"][fallback_rows]

                # Predict on test data
                test_preds[learner_name][slice_name] += model.predict(test_df[Config.FEATURES])

    # Average test predictions over folds
    for learner_name in test_preds:
        for slice_name in test_preds[learner_name]:
            test_preds[learner_name][slice_name] /= Config.N_FOLDS

    return oof_preds, test_preds, model_slices, models, model_names


In [11]:
def ensemble_and_submit(train_df, oof_preds, test_preds, submission_df):
    learner_ensembles = {}

    for learner_name in oof_preds:
        print(f"\n--- {learner_name.upper()} ---")

        # Compute Pearson correlation for each model slice
        slice_scores = {
            slice_name: pearsonr(train_df[Config.LABEL_COLUMN], oof_preds[learner_name][slice_name])[0]
            for slice_name in oof_preds[learner_name]
        }
        total_score = sum(slice_scores.values())

        # === Simple Average Ensemble ===
        oof_simple = np.mean(list(oof_preds[learner_name].values()), axis=0)
        test_simple = np.mean(list(test_preds[learner_name].values()), axis=0)
        score_simple = pearsonr(train_df[Config.LABEL_COLUMN], oof_simple)[0]

        # === Weighted Average Ensemble (weighted by Pearson correlation) ===
        oof_weighted = sum(
            (slice_scores[slice_name] / total_score) * oof_preds[learner_name][slice_name]
            for slice_name in slice_scores
        )
        test_weighted = sum(
            (slice_scores[slice_name] / total_score) * test_preds[learner_name][slice_name]
            for slice_name in slice_scores
        )
        score_weighted = pearsonr(train_df[Config.LABEL_COLUMN], oof_weighted)[0]

        # Log results
        print(f"Simple Ensemble Pearson:   {score_simple:.4f}")
        print(f"Weighted Ensemble Pearson: {score_weighted:.4f}")

        # Store simple ensemble predictions
        learner_ensembles[learner_name] = {
            "oof_simple": oof_simple,
            "test_simple": test_simple
        }

    # === Final Ensemble Across All Learners ===
    final_oof = np.mean([le["oof_simple"] for le in learner_ensembles.values()], axis=0)

    # Taking the mean as the final prediction 

    # --> Add ridge regression here, not simple equal weighting <-- 
    
    final_test = np.mean([le["test_simple"] for le in learner_ensembles.values()], axis=0)
    final_score = pearsonr(train_df[Config.LABEL_COLUMN], final_oof)[0]

    print(f"\nFINAL Ensemble Across Learners Pearson: {final_score:.4f}")

    # === Save Submission ===
    submission_df["prediction"] = final_test
    submission_df.to_csv("submission.csv", index=False)
    print("Saved: submission.csv")

In [12]:
if __name__ == "__main__":
    train_df, test_df, submission_df = load_data()
    oof_preds, test_preds, model_slices, models, model_names = train_and_evaluate(train_df, test_df)
    ensemble_and_submit(train_df, oof_preds, test_preds, submission_df)

ArrowInvalid: No match for FieldRef.Name(liquidity_imbalance) in bid_qty: double
ask_qty: double
buy_qty: double
sell_qty: double
volume: double
X1: double
X2: double
X3: double
X4: double
X5: double
X6: double
X7: double
X8: double
X9: double
X10: double
X11: double
X12: double
X13: double
X14: double
X15: double
X16: double
X17: double
X18: double
X19: double
X20: double
X21: double
X22: double
X23: double
X24: double
X25: double
X26: double
X27: double
X28: double
X29: double
X30: double
X31: double
X32: double
X33: double
X34: double
X35: double
X36: double
X37: double
X38: double
X39: double
X40: double
X41: double
X42: double
X43: double
X44: double
X45: double
X46: double
X47: double
X48: double
X49: double
X50: double
X51: double
X52: double
X53: double
X54: double
X55: double
X56: double
X57: double
X58: double
X59: double
X60: double
X61: double
X62: double
X63: double
X64: double
X65: double
X66: double
X67: double
X68: double
X69: double
X70: double
X71: double
X72: double
